# Vorbereitung Google Dev Board

Lesen und folgen Sie Kapitel 3-4 des folgenden Links:

https://coral.ai/docs/dev-board/get-started/#install-mdt

Wenn Sie sich mit ihrem Board verbunden haben, können Sie mit dem alias "coral"
die vordefinierte Pythonumgebung aktivieren.
Mit:

    jupyter-lab --ip=xxx.xxx.xxx.xxx
    
Können Sie jupyter-lab starten. Die Ip-adresse bekommen Sie mittels "ip a" heraus.
Nun sollten Sie einen Link in der Kommandozeile finden, den Sie auf ihrem lokalen Rechner
in den Browser eingeben können. 

### Achtung

Das Google Dev Board ist an einen ssh key gebunden. Hieraus resultiert, dass sich nur ein Rechner per ssh verbinden kann.
Bitte löschen Sie ihren Key nach der Benutzung des Boards mit dem Befehl  
    
    rm ~/.ssh/authorized_keys
    
oder installieren einen cronjob in dem Sie 

    (crontab -l; echo "*/15 * * * * rm /home/mendel/.ssh/authorized_keys")|awk '!x[$0]++'|crontab -
    
in die Kommandozeile des Google Dev Boards eingeben

# Quantisierung

Bevor wir ein Netzwerk auf dem Dev-Board ausführen können, müssen wir Besonderheiten beachten. 
Die Edge-TPU kann nur quantisierte Netzwerke ausfuehren. 
Wenn Sie ein NN auf ihrem lokalen Rechner trainieren, sind dessen Gewichte typischerweise im float32 format.
Das Dev-Board verlangt Gewichte im uint8 format. Hier muss also eine Quantisierung von 32 bit auf 8 bit stattfinden. 
Sie Muessen ihr Netzwerk also quantisieren. 
Hierbei gibt es zwei Optionen:

1. *Quantization-aware training*:  Trainingsschichten werden mittels ”fake” quantization nodes auf die 8-bit Gewichte der späteren Quantisierung vorbereitet.
Daraus resultiert im allgemeinen eine höhere Genauigkeit im Vergleich zum
post-training.

2. *Full integer post-training quantization*:  Jedes beliebige NN kann nach dem
training quantisiert werden. Es wird allerdings eine "helperfunktion" benötig,
die der Quantisierungsroutine einen Beispielinput übergibt.

Wir verwenden hier Variante 2


In [ ]:
# Laden des Models!

Modelname = "Cnn"
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model(Modelname)

model.summary()

 
 Wir benötigen nur ein paar Beispieldaten für die Quantisierung
 Hierbei wird der Prozess des Vorhersagens ein paar mal durchgespielt
 und die Gewichte demensprechend beschnitten
 Am Ende erhalten wir ein Model, das Gewichte im uint8 Format besitzt
 Vorteil hierbei ist, dass die Vorhersage schneller abgewickelt wird (Auf Kosten der Genauigkeit)



In [ ]:
# Quantisierung

modelname_quant = "model_quant.tflite"

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
X_train=x_train[0:100] / 255
X_train=np.reshape(X_train, (X_train.shape[0],1,28,28,1))
del x_train, y_train, x_test, y_test




def representative_data_gen_CNN():
    for (label,img) in enumerate(X_train):     
        yield [img.astype(np.float32)]


# Kompiliertes und trainiertes Modell uebergeben
converter = tf.lite.TFLiteConverter.from_keras_model(model)


# Quantisierung aktivieren
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Hier den Generator einbinden
converter.representative_dataset = representative_data_gen_CNN


# Wirf Fehlermeldung, falls Operation nicht convertiert werden kann
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


# Quantisierung auf int8 setzen
converter.target_spec.supported_types = [tf.int8]


# Input/Output auf uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8


# Konvertieren
tflite_model = converter.convert()


# Speichern
with open(modelname_quant, 'wb') as f:
    f.write(tflite_model)


## Edge-TPU-Compiler

Für die Edge-TPU reicht dies allerdings noch nicht.

Das Netzwerk muss erst noch durch den TPU-Compiler übersetzte werden.

Leider gibt es den Compiler nur für Debian-basierte Betriebssysteme.

Die Installation und Ausführung ist hier beschrieben:


https://coral.ai/docs/edgetpu/compiler/#system-requirements






## Alternative Dockerimage:


Falls Sie kein Debian-basiertes System besitzen können Sie hierfür Docker verwenden.
Installieren Sie Docker und führen Sie folgende Befehle in ihrem Notebook aus


In [ ]:
run_inside_docker = "\"cd data && edgetpu_compiler {modelname}\"".format(modelname = modelname_quant)

!docker run -v $(pwd):/data -it sichrist/edge_tpu_compiler bash -c {run_inside_docker}

---

Das Modell kann innerhalb des Notebooks mit folgendem Befehl auf das Board gepusht werden:


---

In [ ]:

!mdt push {modelname_to_push} Pfad/zum/Verzeichnis/


 
 
 Um Ihr Netzwerk auf dem Google Dev Board auszuführen, können Sie dieses 
 [Notebook](https://github.com/oduerr/ki/blob/main/exercise_2/Mnist_Coral.ipynb) verwenden.
 
 Das Notebook finden Sie auch im Heimverzeichnis des Dev Boards